In [1]:
#This notebook also uses the `(some) LaTeX environments for Jupyter`
#https://github.com/ProfFan/latex_envs wich is part of the
#jupyter_contrib_nbextensions package

from myhdl import *
from myhdlpeek import Peeker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sympy import *
init_printing()

import itertools

#EE drawing tools in python from https://cdelker.bitbucket.io/SchemDraw/
import SchemDraw as schem
import SchemDraw.elements as e
import SchemDraw.logic as l

#https://github.com/jrjohansson/version_information
%load_ext version_information
%version_information myhdl, myhdlpeek, numpy, pandas, matplotlib, sympy, itertools, SchemDraw

Software,Version
Python,3.6.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,6.2.1
OS,Linux 4.15.0 30 generic x86_64 with debian stretch sid
myhdl,0.10
myhdlpeek,0.0.6
numpy,1.13.3
pandas,0.23.3
matplotlib,2.1.0
sympy,1.1.2.dev
itertools,The 'itertools' distribution was not found and is required by the application


In [2]:
#helper  functions to read in the .v and .vhd generated files into python
def VerilogTextReader(loc, printresult=True):
    with open(f'{loc}.v', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***Verilog modual from {loc}.v***\n\n', VerilogText)
    return VerilogText

def VHDLTextReader(loc, printresult=True):
    with open(f'{loc}.vhd', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***VHDL modual from {loc}.vhd***\n\n', VerilogText)
    return VerilogText

def ConstraintXDCTextReader(loc, printresult=True):
    with open(f'{loc}.xdc', 'r') as xdcText:
        ConstraintText=xdcText.read()
    if printresult:
        print(f'***Constraint file from {loc}.xdc***\n\n', ConstraintText)
    return ConstraintText

In [3]:
def TruthTabelGenrator(BoolSymFunc):
    """
    Function to generate a truth table from a sympy boolian expression
    BoolSymFunc: sympy boolian expression
    return TT: a Truth table stored in a pandas dataframe
    """
    colsL=sorted([i for i in list(BoolSymFunc.rhs.atoms())], key=lambda x:x.sort_key())
    colsR=sorted([i for i in list(BoolSymFunc.lhs.atoms())], key=lambda x:x.sort_key())
    bitwidth=len(colsL)
    cols=colsL+colsR; cols
    
    TT=pd.DataFrame(columns=cols, index=range(2**bitwidth))
    
    for i in range(2**bitwidth):
        inputs=[int(j) for j in list(np.binary_repr(i, bitwidth))]
        outputs=BoolSymFunc.rhs.subs({j:v for j, v in zip(colsL, inputs)})
        inputs.append(int(bool(outputs)))
        TT.iloc[i]=inputs
    
    return TT
    
    
    

# Multiplexer

## 2 Channel Input:1 Channel Output multiplexer in compontirical logic

### Sympy Expression

In [4]:
a, b, s, y=symbols('a, b, s, y')
yEq=Eq(y, (~s&a) |(s&b) ); yEq

In [5]:
TruthTabelGenrator(yEq)

,a,b,s,y
0,0,0,0,0
1,0,0,1,0
2,0,1,0,0
3,0,1,1,1
4,1,0,0,1
5,1,0,1,0
6,1,1,0,1
7,1,1,1,1


In [6]:
yEqN=lambdify([a, b, s], yEq.rhs, dummify=False)
SystmaticVals=np.array(list(itertools.product([0,1], repeat=3)))
SystmaticVals, yEqN(SystmaticVals[:, 0], SystmaticVals[:, 1], SystmaticVals[:, 2]).astype(int)

(array([[0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1]]), array([0, 0, 0, 1, 1, 0, 1, 1]))

### myHDL Module

In [7]:
@block
def MUX2_1_Combo(a, b, s, y):
    """
    2:1 Multiblexer written in full combo
    Input:
        a(bool): input channel a
        b(bool): input channel b
        s(bool): channel selection input 
    Output:
        y(bool): ouput
    """
    
    @always_comb
    def logic():
        y.next= (not s and a) or (s and b)
    
    return instances()

### myHDL Testing

In [8]:
#generate systmatic and random test values for AndGate_TB
#stimules inputs X1 and X2
TestLen=10
SystmaticVals=list(itertools.product([0,1], repeat=3))
bTVs=np.array([i[2] for i in SystmaticVals]).astype(int)
np.random.seed(15)
bTVs=np.append(bTVs, np.random.randint(0,2, TestLen)).astype(int)

aTVs=np.array([i[1] for i in SystmaticVals]).astype(int)
#the random genrator must have a differint seed beween each generation
#call in order to produce differint values for each call
np.random.seed(16)
aTVs=np.append(aTVs, np.random.randint(0,2, TestLen)).astype(int)

sTVs=np.array([i[0] for i in SystmaticVals]).astype(int)
#the random genrator must have a differint seed beween each generation
#call in order to produce differint values for each call
np.random.seed(17)
sTVs=np.append(sTVs, np.random.randint(0,2, TestLen)).astype(int)


TestLen=len(aTVs)
SystmaticVals, sTVs, bTVs, aTVs, TestLen

([(0, 0, 0),
  (0, 0, 1),
  (0, 1, 0),
  (0, 1, 1),
  (1, 0, 0),
  (1, 0, 1),
  (1, 1, 0),
  (1, 1, 1)],
 array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]),
 array([0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0]),
 18)

In [9]:
Peeker.clear()
a=Signal(bool(0)); Peeker(a, 'a')
b=Signal(bool(0)); Peeker(b, 'b')
s=Signal(bool(0)); Peeker(s, 's')
y=Signal(bool(0)); Peeker(y, 'y')

DUT=MUX2_1_Combo(a, b, s, y)

def MUX2_1_Combo_TB():
    """
    myHDL only testbench for module `MUX2_1_Combo`
    """
    
    @instance
    def stimules():
        for i in range(TestLen):
            a.next=int(aTVs[i])
            b.next=int(bTVs[i])
            s.next=int(sTVs[i])

            yield delay(1)
        
        raise StopSimulation()
        
    return instances()

sim=Simulation(DUT, MUX2_1_Combo_TB(), *Peeker.instances()).run()            

In [10]:
Peeker.to_wavedrom('s', 'a', 'b','y')

In [11]:
MUX2_1_ComboData=Peeker.to_dataframe()
MUX2_1_ComboData=MUX2_1_ComboData[['s', 'a', 'b','y']]
MUX2_1_ComboData

,s,a,b,y
0,0,0,0,0
1,0,0,1,0
2,0,1,0,1
3,0,1,1,1
4,1,0,0,0
5,1,0,1,1
6,1,1,0,0
7,1,1,1,1
8,1,1,0,0
9,1,0,1,1


In [12]:
MUX2_1_ComboData['yRef']=MUX2_1_ComboData.apply(lambda row:yEqN(row['a'], row['b'], row['s']), axis=1).astype(int)
MUX2_1_ComboData

,s,a,b,y,yRef
0,0,0,0,0,0
1,0,0,1,0,0
2,0,1,0,1,1
3,0,1,1,1,1
4,1,0,0,0,0
5,1,0,1,1,1
6,1,1,0,0,0
7,1,1,1,1,1
8,1,1,0,0,0
9,1,0,1,1,1


In [13]:
Test=(MUX2_1_ComboData['y']==MUX2_1_ComboData['yRef']).all()
print(f'Module `MUX2_1_Combo` works as exspected: {Test}')

Module `MUX2_1_Combo` works as exspected: True


### Verilog Conversion

In [14]:
DUT.convert()
VerilogTextReader('MUX2_1_Combo');

***Verilog modual from MUX2_1_Combo.v***

 // File: MUX2_1_Combo.v
// Generated by MyHDL 0.10
// Date: Sat Sep 15 21:15:41 2018


`timescale 1ns/10ps

module MUX2_1_Combo (
    a,
    b,
    s,
    y
);
// Input:
//     a(bool)
//     b(bool)
//     s(bool)
// Output:
//     y(bool)

input a;
input b;
input s;
output y;
wire y;





assign y = (((!s) && a) || (s && b));

endmodule



\begin{figure}
\centerline{\includegraphics[width=10cm]{MUX2_1_Combo_RTL.png}}
\caption{\label{fig:M21CRTL} MUX2_1_Combo RTL schematic; Xilinx Vivado 2017.4}
\end{figure}

\begin{figure}
\centerline{\includegraphics[width=10cm]{MUX2_1_Combo_SYN.png}}
\caption{\label{fig:M21CSYN} MUX2_1_Combo Synthesized Schematic; Xilinx Vivado 2017.4}
\end{figure}

\begin{figure}
\centerline{\includegraphics[width=10cm]{MUX2_1_Combo_IMP.png}}
\caption{\label{fig:M21CIMP} MUX2_1_Combo Implementated Schematic; Xilinx Vivado 2017.4}
\end{figure}

### myHDL to Verilog Testbench

In [15]:
#create BitVectors for AndGate_TBV
aTVs=intbv(int(''.join(aTVs.astype(str)), 2))[TestLen:]
bTVs=intbv(int(''.join(bTVs.astype(str)), 2))[TestLen:]
sTVs=intbv(int(''.join(sTVs.astype(str)), 2))[TestLen:]

aTVs, bin(aTVs), bTVs, bin(bTVs), sTVs, bin(sTVs)

(intbv(52982),
 '1100111011110110',
 intbv(87399),
 '10101010101100111',
 intbv(16277),
 '11111110010101')

In [16]:
@block
def MUX2_1_Combo_TBV():
    """
    myHDL -> Verilog testbench for module `MUX2_1_Combo`
    """
    a=Signal(bool(0))
    b=Signal(bool(0))
    s=Signal(bool(0))
    y=Signal(bool(0))
    
    @always_comb
    def print_data():
        print(a, b, s, y)
    
    #Test Signal Bit Vectors
    aTV=Signal(aTVs)
    bTV=Signal(bTVs)
    sTV=Signal(sTVs)


    DUT=MUX2_1_Combo(a, b, s, y)

    @instance
    def stimules():
        for i in range(TestLen):
            a.next=int(aTV[i])
            b.next=int(bTV[i])
            s.next=int(sTV[i])
            yield delay(1)
        
        raise StopSimulation()
    return instances()

TB=MUX2_1_Combo_TBV()
TB.convert(hdl="Verilog", initial_values=True)
VerilogTextReader('MUX2_1_Combo_TBV');         

<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
***Verilog modual from MUX2_1_Combo_TBV.v***

 // File: MUX2_1_Combo_TBV.v
// Generated by MyHDL 0.10
// Date: Sat Sep 15 21:15:41 2018


`timescale 1ns/10ps

module MUX2_1_Combo_TBV (

);
// myHDL -> Verilog testbench for module `AndGate`


reg a = 0;
reg b = 0;
reg s = 0;
wire y;
wire [17:0] aTV;
wire [17:0] bTV;
wire [17:0] sTV;

assign aTV = 18'd52982;
assign bTV = 18'd87399;
assign sTV = 18'd16277;


always @(y, a, b, s) begin: MUX2_1_COMBO_TBV_PRINT_DATA
    $write("%h", a);
    $write(" ");
    $write("%h", b);
    $write(" ");
    $write("%h", s);
    $write(" ");
    $write("%h", y);
    $write("\n");
end



assign y = (((!s) && a) || (s && b));


initial begin: MUX2_1_COMBO_TBV_STIMULES
    integer i;
    for (i=0; i<18; i=i+1) begin
        a <= aTV[i];
        b <= bTV[

/home/iridium/anaconda3/lib/python3.6/site-packages/myhdl/conversion/_toVerilog.py:349: ToVerilogWarning: Signal is not driven: aTV
  category=ToVerilogWarning
/home/iridium/anaconda3/lib/python3.6/site-packages/myhdl/conversion/_toVerilog.py:349: ToVerilogWarning: Signal is not driven: bTV
  category=ToVerilogWarning
/home/iridium/anaconda3/lib/python3.6/site-packages/myhdl/conversion/_toVerilog.py:349: ToVerilogWarning: Signal is not driven: sTV
  category=ToVerilogWarning


### PYNQ-Z1 Deployment

#### Board Circuit

#### Board Constraint

In [18]:
ConstraintXDCTextReader('MUX2_1_Combo');

***Constraint file from MUX2_1_Combo.xdc***

 ## PYNQ-Z1 Constraint File for MUX2_1_Combo
## Based on https://github.com/Xilinx/PYNQ/blob/master/boards/Pynq-Z1/base/vivado/constraints/base.xdc


## Switches
set_property -dict {PACKAGE_PIN M20 IOSTANDARD LVCMOS33} [get_ports {s}]; ##SW0

## Buttons
set_property -dict {PACKAGE_PIN D19 IOSTANDARD LVCMOS33} [get_ports {b}]; ## BT0
set_property -dict {PACKAGE_PIN D20 IOSTANDARD LVCMOS33} [get_ports {a}]; ##BT1


## LEDs
set_property -dict {PACKAGE_PIN R14 IOSTANDARD LVCMOS33} [get_ports {y}]; ## Led 0





#### Video of Deployment
MUX2_1_Combo myHDL PYNQ-Z1 ([YouTube]())

## 4:1

In [17]:
a, b, c, d, s0, s1, y=symbols('a, b, c, d, s_0, s_1, y')
yEq=Eq(y, (~s0&~s1&a) | (s0&~s1&b)| (~s0&s1&c)|(s0&s1&d)); yEq